In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [105]:
df = pd.read_csv('../data/data.tsv', sep='\t',error_bad_lines=False)

b'Skipping line 83032: expected 6 fields, saw 7\n'
b'Skipping line 154657: expected 6 fields, saw 7\n'
b'Skipping line 323916: expected 6 fields, saw 7\n'
/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [106]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363192 entries, 0 to 363191
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            363192 non-null  object 
 1   qid1          363192 non-null  object 
 2   qid2          363185 non-null  float64
 3   question1     363181 non-null  object 
 4   question2     363180 non-null  object 
 5   is_duplicate  363180 non-null  float64
dtypes: float64(2), object(4)
memory usage: 16.6+ MB


In [108]:
df = df.dropna(axis=0, subset=('question1','question2','is_duplicate' ))

In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 363177 entries, 0 to 363191
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            363177 non-null  object 
 1   qid1          363177 non-null  object 
 2   qid2          363177 non-null  float64
 3   question1     363177 non-null  object 
 4   question2     363177 non-null  object 
 5   is_duplicate  363177 non-null  float64
dtypes: float64(2), object(4)
memory usage: 19.4+ MB


In [110]:
#lower casing
def lower_case(text):
    return text.lower()


#remove numbers
def remove_numbers(text):
    output = re.sub(r'\d+', '', text)
    return output

# remove punctuation
import string
def remove_punctuation(text):
    text_p = "".join([char for char in text if char not in string.punctuation])
    return text_p

#tokenize text

import nltk
nltk.download('punkt')
from nltk import word_tokenize

def tokenize(text):
    words = word_tokenize(text)
    return words

#remove stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

def remove_stopwords(text):
    filtered_words = [word for word in text if word not in stop_words]
    return filtered_words

#stemming
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def stem_words(text):
    stemmed = [stemmer.stem(word) for word in text]
    return stemmed

# Lemmatize with POS Tag
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')


from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    lemmatized = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in text]
    return lemmatized

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [111]:
class Preprocessor():
    def __init__(self):
        pass
    
    def preprocess(self,df_column, steps):
        self.df_column = df_column
        self.steps = steps
    
        if 'lower_case' in self.steps:
            self.df_column = self.df_column.apply(lambda x: lower_case(x))
        
        
        if 'remove_numbers' in self.steps:
            self.df_column = self.df_column.apply(lambda x: remove_numbers(x))
        
        if 'remove_punctuation' in self.steps:
            self.df_column = self.df_column.apply(lambda x: remove_punctuation(x))           
        
        if 'tokenize' in self.steps:
            self.df_column = self.df_column.apply(lambda x: tokenize(x))
        
        if 'stopwords' in self.steps:
            self.df_column = self.df_column.apply(lambda x: remove_stopwords(x))
        
        if 'stemming' in self.steps:
            self.df_column = self.df_column.apply(lambda x: stem_words(x))
            
        if 'lemmatize' in self.steps:
            self.df_column = self.df_column.apply(lambda x: lemmatize(x))    
            
        return self.df_column

In [112]:
steps = ['lower_case','remove_numbers','remove_punctuation',
        'tokenize','stopwords','lemmatize']
processor = Preprocessor()
df['question1'] = processor.preprocess(df['question1'] ,steps)

In [113]:
df['question2'] = processor.preprocess(df['question2'] ,steps)

In [114]:
df.reset_index(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363177 entries, 0 to 363176
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   index         363177 non-null  int64  
 1   id            363177 non-null  object 
 2   qid1          363177 non-null  object 
 3   qid2          363177 non-null  float64
 4   question1     363177 non-null  object 
 5   question2     363177 non-null  object 
 6   is_duplicate  363177 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 19.4+ MB


In [115]:
df.head()

,index,id,qid1,qid2,question1,question2,is_duplicate
0,0,133273,213221,213222.0,"[life, math, student, could, describe, experie...","[level, prepration, enough, exam, jlpt]",0.0
1,1,402555,536040,536041.0,"[control, horny, emotion]","[control, horniness]",1.0
2,2,360472,364011,490273.0,"[cause, stool, color, change, yellow]","[cause, stool, come, little, ball]",0.0
3,3,150662,155721,7256.0,"[one, mbbs]",[mbbs],1.0
4,4,183004,279958,279959.0,"[find, power, outlet, laptop, melbourne, airport]","[would, second, airport, sydney, australia, ne...",0.0


In [116]:
q2vocab = df['question2'].tolist()
q2vocab = [item for sublist in q2vocab for item in sublist]
q2vocab = list(set(q2vocab))
len(q2vocab)

56588

In [117]:
#define function 
def get_Doc_ids(vocabulary_list,dataframe,tokens_col_name, id_col_name):
    Doc_ids = {}      #initialise empty document frequency dictionary
    
    #for each word in vocabulary, create key:value pair, with an empty list as value
    for word in vocabulary_list:
        Doc_ids[word] = [] 
        
    #for each word in DF, add the id of question2 where it appears. Create a set to avoid duplicate ids
    for i in range(len(dataframe)):
        for word in dataframe.loc[i,tokens_col_name]:
            Doc_ids[word].append(dataframe[id_col_name][i])
    for word in Doc_ids.keys():
        Doc_ids[word] = list(set(Doc_ids[word]))
        #DF[word]  =len(DF[word])
        
    return Doc_ids

Doc_ids = get_Doc_ids(q2vocab,df,'question2','qid2')
list(Doc_ids.items())[:5]

[('cupcake',
  [85889.0,
   36228.0,
   11269.0,
   195462.0,
   359051.0,
   2574.0,
   2575.0,
   209425.0,
   442524.0,
   3487.0,
   43042.0,
   507955.0,
   570.0,
   571.0,
   151358.0,
   99651.0,
   135372.0,
   88918.0,
   207831.0,
   207830.0,
   18273.0,
   16227.0,
   16228.0,
   5867.0,
   227955.0,
   122997.0,
   32373.0,
   40441.0]),
 ('galway', [346536.0, 185355.0, 250990.0]),
 ('declassify', [29364.0]),
 ('peach',
  [322528.0,
   241384.0,
   516055.0,
   382099.0,
   456023.0,
   112922.0,
   18492.0,
   532671.0]),
 ('auntuncle', [321940.0])]

In [118]:
#dictionary - word: list of doc_ids
Word_Doc_ids = {}
for word in q2vocab:
    Word_Doc_ids[word] = []
for i in range(len(df)):
    for word in df.loc[i,'question2']:
        Word_Doc_ids[word].append(df.loc[i,'qid2'])

#create set to drop duplicate values of doc_ids
for word in q2vocab:
    Word_Doc_ids[word] = list(set(Word_Doc_ids[word]))
# for i in range(len(q2vocab)):
#     if q2vocab[i] in df.loc[i,'question2']:
#         Word_Doc_ids[q2vocab[i]].append(df.loc[i,'qid2'])
# for word in q2vocab:        
#     Word_Doc_ids[word] = list(set(Word_Doc_ids[word]))        
    

In [119]:
list(Word_Doc_ids.items())[:2]

[('cupcake',
  [85889.0,
   36228.0,
   11269.0,
   195462.0,
   359051.0,
   2574.0,
   2575.0,
   209425.0,
   442524.0,
   3487.0,
   43042.0,
   507955.0,
   570.0,
   571.0,
   151358.0,
   99651.0,
   135372.0,
   88918.0,
   207831.0,
   207830.0,
   18273.0,
   16227.0,
   16228.0,
   5867.0,
   227955.0,
   122997.0,
   32373.0,
   40441.0]),
 ('galway', [346536.0, 185355.0, 250990.0])]

In [120]:
#create Inverse Document_frequency (IDF) dictionary
N = df['qid2'].nunique()
print(N)

IDF = {}
for word in Doc_ids.keys():
    IDF[word] = N/(len(Word_Doc_ids[word])+1)
list(IDF.items())[:5]    

273121


[('cupcake', 9417.965517241379),
 ('galway', 68280.25),
 ('declassify', 136560.5),
 ('peach', 30346.777777777777),
 ('auntuncle', 136560.5)]

In [121]:
df['q2_len'] = df['question2'].apply(lambda x: len(x) )
df.head(2)

,index,id,qid1,qid2,question1,question2,is_duplicate,q2_len
0,0,133273,213221,213222.0,"[life, math, student, could, describe, experie...","[level, prepration, enough, exam, jlpt]",0.0,5
1,1,402555,536040,536041.0,"[control, horny, emotion]","[control, horniness]",1.0,2


In [122]:
#dictionary - doc_id: words
doc_words = dict(zip(df['qid2'],df['question2']))
# tf_lists = {}
# for word in q2vocab:
#     word_tf = []
#     for doc in Doc_ids[word]:
#         tf = doc_words[doc].count(word)/len(doc_words[doc])
#         word_tf.append(tf)
#     tf_lists[word] = word_tf 
    
# list(tf_lists.items())[:5] 
list(doc_words.items())[:2]

[(213222.0, ['level', 'prepration', 'enough', 'exam', 'jlpt']),
 (536041.0, ['control', 'horniness'])]

In [123]:
# Create a dictionary where keys are words in the vocab, 
#values are dictionary of key:values as doc_id:tf for corresponding word
#nested dictionary - word: {{doc_id1: tf}, {doc_id2: tf}, ....}
inverted_tf = {}
for word in q2vocab:
    word_tf = {}
    for doc in Word_Doc_ids[word]:
        tf = doc_words[doc].count(word)/len(doc_words[doc])
        word_tf[doc] = tf
    inverted_tf[word] = word_tf   

In [124]:
list(inverted_tf.items())[0]

('cupcake',
 {85889.0: 0.1111111111111111,
  36228.0: 0.125,
  11269.0: 0.16666666666666666,
  195462.0: 0.1111111111111111,
  359051.0: 0.16666666666666666,
  2574.0: 0.16666666666666666,
  2575.0: 0.1111111111111111,
  209425.0: 0.25,
  442524.0: 0.16666666666666666,
  3487.0: 0.16666666666666666,
  43042.0: 0.125,
  507955.0: 0.2,
  570.0: 0.14285714285714285,
  571.0: 0.2,
  151358.0: 0.16666666666666666,
  99651.0: 0.25,
  135372.0: 0.14285714285714285,
  88918.0: 0.14285714285714285,
  207831.0: 0.2,
  207830.0: 0.3333333333333333,
  18273.0: 0.16666666666666666,
  16227.0: 0.3333333333333333,
  16228.0: 0.3333333333333333,
  5867.0: 0.16666666666666666,
  227955.0: 0.16666666666666666,
  122997.0: 0.14285714285714285,
  32373.0: 0.14285714285714285,
  40441.0: 0.16666666666666666})

In [125]:
# tfidf_first = []
# first = list(inverted_tf['header'].items()) 
# for tup in first:
#     tup = list(tup)
#     tfidf = tup[1]*np.log10(IDF['header'])
#     tup[1] = tfidf
#     tfidf_first.append(tup)
# tfidf_first

In [126]:
# Create a dictionary where keys are words in the vocab, 
#values are dictinary of key:values as doc_id:tfidf for corresponding word
#nested dictionary - word: {{doc_id1: tf}, {doc_id2: tf}, ....}
inverted_tfidf = {}
for word in q2vocab:
    doc_tfidfs = []
    for tup in list(inverted_tf[word].items()):
        tup = list(tup)
        tfidf = tup[1]*np.log10(IDF[word])
        tup[1] = tfidf
        doc_tfidfs.append(tup)
    inverted_tfidf[word] = doc_tfidfs  

# for i in range(len(inverted_tfidf)):
#     inverted_tfidf[q2vocab[i]] = [dict([[x[0],x[1]]])
#                                 for x in inverted_tfidf[q2vocab[i]]]
     

In [127]:
list(inverted_tfidf.items())[0]

('cupcake',
 [[85889.0, 0.44155078844356815],
  [36228.0, 0.4967446369990142],
  [11269.0, 0.6623261826653523],
  [195462.0, 0.44155078844356815],
  [359051.0, 0.6623261826653523],
  [2574.0, 0.6623261826653523],
  [2575.0, 0.44155078844356815],
  [209425.0, 0.9934892739980284],
  [442524.0, 0.6623261826653523],
  [3487.0, 0.6623261826653523],
  [43042.0, 0.4967446369990142],
  [507955.0, 0.7947914191984228],
  [570.0, 0.5677081565703019],
  [571.0, 0.7947914191984228],
  [151358.0, 0.6623261826653523],
  [99651.0, 0.9934892739980284],
  [135372.0, 0.5677081565703019],
  [88918.0, 0.5677081565703019],
  [207831.0, 0.7947914191984228],
  [207830.0, 1.3246523653307045],
  [18273.0, 0.6623261826653523],
  [16227.0, 1.3246523653307045],
  [16228.0, 1.3246523653307045],
  [5867.0, 0.6623261826653523],
  [227955.0, 0.6623261826653523],
  [122997.0, 0.5677081565703019],
  [32373.0, 0.5677081565703019],
  [40441.0, 0.6623261826653523]])

In [128]:
for i in range(len(inverted_tfidf)):
    inverted_tfidf[q2vocab[i]] = {x[0]:x[1] for x in inverted_tfidf[q2vocab[i]]}

In [129]:
list(inverted_tfidf.items())[0]

('cupcake',
 {85889.0: 0.44155078844356815,
  36228.0: 0.4967446369990142,
  11269.0: 0.6623261826653523,
  195462.0: 0.44155078844356815,
  359051.0: 0.6623261826653523,
  2574.0: 0.6623261826653523,
  2575.0: 0.44155078844356815,
  209425.0: 0.9934892739980284,
  442524.0: 0.6623261826653523,
  3487.0: 0.6623261826653523,
  43042.0: 0.4967446369990142,
  507955.0: 0.7947914191984228,
  570.0: 0.5677081565703019,
  571.0: 0.7947914191984228,
  151358.0: 0.6623261826653523,
  99651.0: 0.9934892739980284,
  135372.0: 0.5677081565703019,
  88918.0: 0.5677081565703019,
  207831.0: 0.7947914191984228,
  207830.0: 1.3246523653307045,
  18273.0: 0.6623261826653523,
  16227.0: 1.3246523653307045,
  16228.0: 1.3246523653307045,
  5867.0: 0.6623261826653523,
  227955.0: 0.6623261826653523,
  122997.0: 0.5677081565703019,
  32373.0: 0.5677081565703019,
  40441.0: 0.6623261826653523})

In [217]:
queries = df[df['is_duplicate'] == 1][:100]
queries = queries[['question1','qid2','question2', 'is_duplicate']]
queries = queries.reset_index()
queries.head()

,index,question1,qid2,question2,is_duplicate
0,1,"[control, horny, emotion]",536041.0,"[control, horniness]",1.0
1,3,"[one, mbbs]",7256.0,[mbbs],1.0
2,7,"[best, self, help, book, read, change, life]",787.0,"[top, self, help, book, read]",1.0
3,11,"[hillary, clinton, policy, towards, india, bec...",177376.0,"[hilary, clinton, policy, towards, india, beco...",1.0
4,13,"[best, book, study, tensor, general, relativit...",22333.0,"[best, book, tensor, calculus]",1.0


In [131]:
len(queries[:100])

100

In [219]:
from collections import Counter
def ranked_docs(query, inverted_file):
    counter_objects = []
    for word in query:
        if word in inverted_file.keys():
            counter_objects.append(Counter(inverted_file[word]))
            combined = sum(counter_objects,Counter())
            ranked_docs = sorted(combined, key = combined.get, reverse=True)
    return ranked_docs 
query = queries.loc[0,'question1']
#ranked_docs(query,inverted_tfidf)
print(query)
ranked_docs(query,inverted_tfidf)[:5]

['control', 'horny', 'emotion']


[355257.0, 344642.0, 348571.0, 42047.0, 132587.0]

In [214]:
def ranked_docs2(query,inverted_file):
    counter_objects = []
    for word in query:
        if word in inverted_file.keys():
            counter_objects.append(inverted_file[word])
    c = Counter()
    for cob in counter_objects:
        c.update(cob)
    ranked_docs = sorted(c, key = c.get, reverse=True) 
    return ranked_docs,counter_objects

ranked_docs2,_ = ranked_docs2(query,inverted_tfidf)
ranked_docs2[:5]

[355257.0, 344642.0, 348571.0, 42047.0, 132587.0]

In [215]:
counter_objects = []
for word in query:
    if word in inverted_tfidf.keys():
        counter_objects.append(Counter(inverted_tfidf[word]))
        combined = sum(counter_objects,Counter())
        ranked_docs = sorted(combined, key = combined.get, reverse=True)
ranked_docs[:5]


[355257.0, 344642.0, 348571.0, 42047.0, 132587.0]

In [203]:
counter_objects = []
counter_objects.append(inverted_tfidf['control'])
counter_objects.append(inverted_tfidf['horny'])
counter_objects.append(inverted_tfidf['emotion'])
counter_objects[0]
c = Counter()
for cob in counter_objects:
    c.update(cob)
ranked_docs = sorted(c, key = c.get, reverse=True)
ranked_docs[:5]

[355257.0, 344642.0, 348571.0, 42047.0, 132587.0]

In [205]:
df[df['qid2'] == 355257.0]

,index,id,qid1,qid2,question1,question2,is_duplicate,q2_len
322413,322425,242931,310574,355257.0,"[always, horny]",[horny],1.0,1


In [220]:
for i in range(len(queries)):
    query = queries.loc[i,'question1']
    ranked = ranked_docs(query, inverted_tfidf)
    if queries.loc[i,'qid2'] in ranked[:2]:
        queries.loc[i,'top5_pred'] = 1.0
        queries.loc[i,'top2_pred'] = 1.0
    elif queries.loc[i,'qid2'] in ranked[:5]:
        queries.loc[i,'top5_pred'] = 1.0
        queries.loc[i,'top2_pred'] = 0.0
    else:
        queries.loc[i,'top5_pred'] = 0.0
        queries.loc[i,'top2_pred'] = 0.0
    
    
queries.head()

,index,question1,qid2,question2,is_duplicate,top5_pred,top2_pred
0,1,"[control, horny, emotion]",536041.0,"[control, horniness]",1.0,0.0,0.0
1,3,"[one, mbbs]",7256.0,[mbbs],1.0,1.0,0.0
2,7,"[best, self, help, book, read, change, life]",787.0,"[top, self, help, book, read]",1.0,0.0,0.0
3,11,"[hillary, clinton, policy, towards, india, bec...",177376.0,"[hilary, clinton, policy, towards, india, beco...",1.0,0.0,0.0
4,13,"[best, book, study, tensor, general, relativit...",22333.0,"[best, book, tensor, calculus]",1.0,0.0,0.0


In [221]:
from sklearn.metrics import accuracy_score
accuracy_score(queries['is_duplicate'],queries['top2_pred'])

0.18

In [222]:
accuracy_score(queries['is_duplicate'],queries['top5_pred'])

0.31

In [151]:
query2 = queries.loc[2,'question1']
query2

['best', 'self', 'help', 'book', 'read', 'change', 'life']

In [152]:
ranked_docs(query2,inverted_tfidf)[:5]

[203474.0, 157424.0, 26371.0, 125925.0, 110004.0]

In [167]:
inverted_tfidf['help'][203474.0]

2.1539595891485446

In [184]:
counter_objects = []
for word in query2:
    
    if word in inverted_tfidf.keys():
        counter_objects.append(inverted_tfidf[word])

c = Counter()
for cob in counter_objects:
    c.update(cob)
    
#combined = sum(counter_objects,Counter())
ranked_docs = sorted(c, key = c.get, reverse=True)
ranked_docs[:5]
len(counter_objects)

7

In [169]:
counter_objects = []
len(Counter(inverted_tfidf['top']))

1438

In [ ]:
list_dicts = []
for word in query:
    list_dicts.append(Counter(inverted_tfidf[word]))
len(list_dicts)    


In [ ]:
merged_dict = sum(list_dicts, Counter())
max(merged_dict, key=merged_dict.get)

In [ ]:
df[df['qid2'] == 355257.0]

In [ ]:
ranked = sorted(merged_dict, key=merged_dict.get, reverse=True)
ranked[:5]

In [226]:
from sklearn.feature_extraction.text import TfidfVectorizer
train_questions = list(df['question2'])
def dummy(text):
    return text
tf = TfidfVectorizer(tokenizer=dummy,preprocessor=dummy, lowercase=False)
X_train = tf.fit_transform(train_questions)


In [227]:
XtrainT = X_train.T.todok()


In [228]:
XtrainT.shape

(56588, 363177)

In [ ]:
len(set(tuple(i) for i in train_questions))

In [ ]:
N = df['qid2'].unique()
